##### Import needed libraries and set needed options

In [1]:
import altair as alt
import pandas as pd
import numpy as np
import geopandas as gpd
import json

from pandas.io.json import json_normalize
pd.set_option('display.max_colwidth', None)

# Data preparation

## 1. Intro

### This project uses datasets from: https://data.gov.ua/en/dataset/3fcbfe9e-7cec-4f69-b9ca-be49daae2369 <br>
### Download them from that URL, put in "data" folder and then run<br>

### There are **4 files with data**:
* **immunization_covid19_2021_16.11.2021.csv**   - contains records of vaccinations
* **immunization_legal_entities_info.csv**       - contains infromation about institutions of vaccination
* **immunization_vaccine_codes.csv**             - contains information about vaccines
* **opis-atributiv-naboru-danikh.xlsx**          - contains description of attributes for above 3 files

### From opis-atributiv-naboru-danikh.xlsx we get such description of immunization_covid19_2021_16.11.2021.csv columns:

1) **temp_immunization_id** - Тимчасовий ідентифікатор імунізації<br>
2) **legal_entity_id** - Ідентифікатор надавача послуг в ЕСОЗ<br>
3) **division_id** - Ідентифікатор місця надання послуг в ЕСОЗ<br>
4) **status** - Статус запису про вакцинацію<br>
5) **not_given** - Вакцинацію не проведено<br>
6) **vaccine_code** - Код вакцини<br>
7) **immunization_date** - Дата проведення вакцинації<br>
8) **patient_age_group** - Вікова група пацієнта<br>
9) **patient_gender** - Стать пацієнта<br>
10) **manufacturer** - Назва виробника вакцини<br>
11) **lot_number** - Серія (номер партії) вакцини<br>
12) **expiration_date** - Термін придатності вакцини<br>
13) **dose_quantity_unit** - Одиниці виміру введенної дози<br>
14) **dose_quantity_value** - Розмір введенної дози<br>
15) **vaccination_protocols** - Інформація з протоколу НСЗУ<br>
16) **vaccination_protocols.dose_sequence** - Порядковий номер введеної дози<br>
17) **vaccination_protocols.series** - Етап вакцинації<br>
18) **vaccination_protocols.series_doses** - Кількість доз за протоколом<br>
19) **vaccination_protocols.target_diseases** - Хвороба(-би) проти якої проведено вакцинацію<br>
20) **inserted_at** - Дата внесення в ЕСОЗ<br>
21) **updated_at** - Дата оновлення в ЕСОЗ<br>

### From opis-atributiv-naboru-danikh.xlsx we get such description of immunization_legal_entities_info.csv columns:

1) **legal_entity_id** - Ідентифікатор надавача послуг в ЕСОЗ<br>
2) **legal_entity_name** - Назва надавача послуг<br>
3) **legal_entity_edrpou** - Код ЄДРПОУ надавача послуг<br>
4) **care_type** - Тип медичної допомоги<br>
5) **property_type** - Форма власності<br>
6) **legal_entity_email** - Електронна адреса надавача послуг<br>
7) **legal_entity_phone** - Телефон надавача послуг<br>
8) **legal_entity_owner_name** - Керівник<br>
9) **registration_area** - Область реєстрації<br>
10) **registration_settlement** - Населений пункт реєстрації<br>
11) **registration_addresses** - Адреса реєстрації надавача послуг<br>
12) **lng** - Довгота (геокоординати місця реєстрації)<br>
13) **lat** - Широта (геокоординати місця реєстрації)<br>

### From opis-atributiv-naboru-danikh.xlsx we get such description of immunization_vaccine_codes.csv columns:

1) **vaccine_code** - Код вакцини в ЕСОЗ<br>
2) **vaccine_name** - Назва вакцини<br>

In [2]:
# The dataset is too big, you should download it by yourself and put it to "data" folder and run
# Dataset URL: 
# https://data.gov.ua/dataset/4cced549-1a03-4e0b-afbb-461febb26007/resource/f51566ef-dd6a-47f1-8827-6d5e5ca17fda/download/immunization_covid19_2021.zip

df_vaccination_entries = pd.read_csv('./data/immunization_covid19_2021_16.11.2021.csv')

In [3]:
# The dataset is too big, you should download it by yourself and put it to "data" folder and run
# Dataset URL: 
# https://data.gov.ua/dataset/4cced549-1a03-4e0b-afbb-461febb26007/resource/5906aa4e-4974-486f-8477-b4749d98542e/download/immunization_legal_entities_info.csv

df_vaccination_location = pd.read_csv('./data/immunization_legal_entities_info.csv')

In [4]:
# The dataset is too big, you should download it by yourself and put it to "data" folder and run
# Dataset URL: 
# https://data.gov.ua/dataset/4cced549-1a03-4e0b-afbb-461febb26007/resource/b4d8df31-8ecc-4646-a373-95b756fde8ea/download/immunization_vaccine_codes.csv 

df_vaccine_info = pd.read_csv('./data/immunization_vaccine_codes.csv')

In [5]:
geo = gpd.read_file('./data/ukraine.json')   # Just to add region names for merging when visualizing

## 2.1 Dataset Info (immunization_covid19_2021_16.11.2021.csv)

In [6]:
df_vaccination_entries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20889909 entries, 0 to 20889908
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   temp_immunization_id   int64  
 1   legal_entity_id        object 
 2   division_id            object 
 3   status                 object 
 4   not_given              bool   
 5   vaccine_code           object 
 6   immunization_date      object 
 7   patient_age_group      object 
 8   patient_gender         object 
 9   manufacturer           object 
 10  lot_number             object 
 11  expiration_date        object 
 12  dose_quantity_unit     object 
 13  dose_quantity_value    float64
 14  vaccination_protocols  object 
 15  inserted_at            object 
 16  updated_at             object 
dtypes: bool(1), float64(1), int64(1), object(14)
memory usage: 2.5+ GB


In [7]:
# We need vaccination_protocols data, so drop NaN
df_vaccination_entries = df_vaccination_entries[df_vaccination_entries['vaccination_protocols'].notna()]

In [8]:
# We need to convert vaccination_protocols JSON to columns. Firstly we get dict from array
df_1_protocols = df_vaccination_entries['vaccination_protocols']

In [9]:
# We need to convert vaccination_protocols JSON to columns. Firstly we get dict from array
df_1_protocols = pd.json_normalize(df_1_protocols.apply(json.loads))

In [10]:
# We need to convert vaccination_protocols JSON to columns. Secondly we get columns from dict
df_1_protocols = df_1_protocols[0].apply(pd.Series)

In [11]:
df_vaccination_entries['dose_sequence'] = df_1_protocols['dose_sequence']
df_vaccination_entries['series'] = df_1_protocols['series']
df_vaccination_entries['series_doses'] = df_1_protocols['series_doses']

In [12]:
df_vaccination_entries.head(3)

,temp_immunization_id,legal_entity_id,division_id,status,not_given,vaccine_code,immunization_date,patient_age_group,patient_gender,manufacturer,lot_number,expiration_date,dose_quantity_unit,dose_quantity_value,vaccination_protocols,inserted_at,updated_at,dose_sequence,series,series_doses
0,1,35c628e2-c42d-4d0d-b724-33c1b4db39f7,NaN,Запис внесено помилково,False,SarsCov2_nRVv,2021-02-23,41-45 р.,Чоловіча,СЕРУМ ІНСТІТЬЮТ ОФ ІНДІА ПВТ.ЛТД. ІНДІЯ,NaN,NaN,ML,1.0,"[{""dose_sequence"" : null, ""series"" : null, ""series_doses"" : null, ""target_diseases"" : ""Коронавірусна хвороба (COVID-19)""}]",2021-02-23,2021-02-23,None,None,None
1,2,254ff4d8-5e2f-4db6-9cae-38edabe94729,59d84164-9546-4aa9-9773-2a5248b046e9,Запис коректний,False,SarsCov2_nRVv,2021-02-24,46-50 р.,Жіноча,СЕРУМ ІНСТІТЬЮТ ОФ ІНДІА,4120Z027,2021-06-23,ML,1.0,"[{""dose_sequence"" : ""2"", ""series"" : ""1"", ""series_doses"" : ""1"", ""target_diseases"" : ""Коронавірусна хвороба (COVID-19)""}]",2021-02-24,2021-02-24,2,1,1
2,3,254ff4d8-5e2f-4db6-9cae-38edabe94729,59d84164-9546-4aa9-9773-2a5248b046e9,Запис внесено помилково,False,SarsCov2_nRVv,2021-02-24,51-55 р.,Жіноча,СЕРУМ ІНСТІТЬЮТ ОФ ІНДІА,4120Z027,2021-06-23,ML,0.5,"[{""dose_sequence"" : ""2"", ""series"" : null, ""series_doses"" : ""1"", ""target_diseases"" : ""Коронавірусна хвороба (COVID-19)""}]",2021-02-24,2021-03-29,2,None,1


In [14]:
df_vaccination_entries['patient_age_group'].unique()

array(['41-45 р.', '46-50 р.', '51-55 р.', '26-30 р.', '36-40 р.',
       '31-35 р.', '56-60 р.', '21-25 р.', '66-70 р.', '61-65 р.',
       '16-20 р.', '71-75 р.', '76-80 р.', '81-85 р.', '86-00 р.',
       '05-10 р.', '11-15 р.', '91-95 р.', '96+ р.', '0-4 р.',
       'Уточнюється'], dtype=object)

In [15]:
df_vaccination_entries['patient_gender'].unique()

array(['Чоловіча', 'Жіноча', 'Уточнюється'], dtype=object)

In [195]:
# As we can see, there is a lot of stupid data, which we will drop later
print(df_vaccination_entries['dose_sequence'].unique())

[None '2' '1' ... '1564' '12345' nan]


In [194]:
# As we can see, there is a lot of stupid data, which we will rename later for usage or drop
print(df_vaccination_entries['series'].unique()[0:20])

[None '1' 'Імунізація від COVID, 1 хвиля' '1 етап'
 'Ведення першої дози вакцини від COVID-19' 'перший' 'вакцинація '
 '1 ЕТАП' 'Перший' 'Введення першої дози проти ковід 19' 'Другий'
 'перший етап' 'Введення першої дози вакцини від COVID-19' '1-й етап'
 'Первинна' 'Введення першої дози вакцини від COVID-19  '
 'Введення першої дози вакцини від Covid-19'
 'Введення першої дози вакцини від COVID-19 '
 'Введення першої дози вакцини від COVID-19  \n'
 'Ведення першої дози від Covil 19']


In [18]:
# As we can see, there is a lot of stupid data, which we will drop later
df_vaccination_entries['series_doses'].unique()

array([None, '1', '2', '11', '3', '10', '5', '4', '21', '22', '20', '9',
       '25', '7', '12', '6', '8', '13', '14', '15', '16', '17', '19',
       '18', '110', '51', '32', '40', '23', '24', '26', '27', '28', '29',
       '30', '213', '216', '202103003', '52', '38', '1092021', '45',
       '22222', '219', '222', '30092021', '70', '42', '21221', '34',
       '1978', '16082021', '2023', '21062023', '2021', '30112021', '251',
       '255', '212', '16092021', '2222', '50', '37', '211', '4122021',
       '69', '36', '80', '35', '49', '48', '224', '214', '312',
       '31122021', '222222', '31012022', '44', '5072023', '6072023',
       '215', '230', '59', '304', '316', '300', '301', '302', '303',
       '305', '306', '307', '308', '309', '310', '311', '314', '317',
       '318', '319', '320', '321', '322', '323', '324', '325', '326',
       '327', '328', '329', '330', '234', '331', '332', '333', '334',
       '335', '336', '134', '165', '338', '339', '121', '341', '98',
       '340', '342'

In [13]:
df_vaccination_entries.isna().sum()

temp_immunization_id           0
legal_entity_id                0
division_id              1231156
status                         0
not_given                      0
vaccine_code                   0
immunization_date              0
patient_age_group              0
patient_gender                 0
manufacturer                6523
lot_number                 16356
expiration_date            26905
dose_quantity_unit          6688
dose_quantity_value         6585
vaccination_protocols          0
inserted_at                    0
updated_at                     0
dose_sequence              23933
series                     39875
series_doses               22407
dtype: int64

## 2.2 Dataset Info (immunization_legal_entities_info.csv)

In [167]:
df_vaccination_location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1949 entries, 0 to 1948
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   legal_entity_id          1949 non-null   object 
 1   legal_entity_name        1949 non-null   object 
 2   legal_entity_edrpou      1949 non-null   object 
 3   care_type                1949 non-null   object 
 4   property_type            1949 non-null   object 
 5   legal_entity_email       1949 non-null   object 
 6   legal_entity_phone       1949 non-null   object 
 7   legal_entity_owner_name  1949 non-null   object 
 8   registration_area        1949 non-null   object 
 9   registration_settlement  1949 non-null   object 
 10  registration_addresses   1949 non-null   object 
 11  lat                      1946 non-null   float64
 12  lng                      1946 non-null   float64
dtypes: float64(2), object(11)
memory usage: 198.1+ KB


In [168]:
df_vaccination_location.head(3)

,legal_entity_id,legal_entity_name,legal_entity_edrpou,care_type,property_type,legal_entity_email,legal_entity_phone,legal_entity_owner_name,registration_area,registration_settlement,registration_addresses,lat,lng
0,ad229a7d-9bfe-4cf9-9196-4c7fc91f8c5e,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ТУРКІВСЬКИЙ ЦЕНТР ПЕРВИННОЇ МЕДИКО-САНІТАРНОЇ ДОПОМОГИ"" ТУРКІВСЬКОЇ МІСЬКОЇ РАДИ САМБІРСЬКОГО РАЙОНУ ЛЬВІВСЬКОЇ ОБЛАСТІ",42448068,Первинна,Комунальна,jmp7@ukr.net,+380995271332,Яцкуляк Микола Петрович,ЛЬВІВСЬКА,ТУРКА,"Україна, 82500, Львівська обл., Турківський р-н, місто Турка, ВУЛИЦЯ СІЧОВИХ СТРІЛЬЦІВ, будинок 108",49.161703,23.011938
1,ed60aaef-83c5-4a5f-988c-e906822d3731,"КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО ""ЦЕНТР ПЕРВИННОЇ МЕДИКО-САНІТАРНОЇ ДОПОМОГИ"" ОПІШНЯНСЬКОЇ СЕЛИЩНОЇ РАДИ",43772207,Первинна,Комунальна,opishnya_centr_pmsd@ukr.net,+380535342197,Дригоза Олена Іванівна,ПОЛТАВСЬКА,ОПІШНЯ,"Україна, 38164, Полтавська обл., Зіньківський р-н, селище міського типу Опішня, вул.Нова, будинок 54",49.954295,34.590018
2,2e756a68-36ff-4469-95db-0d367b1fbbdb,"ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ ""БОМЕД""",41521487,Первинна,Приватна (без ФОП),bomedmedcentr@gmail.com,+380980103103,Палагнюк Борис Леонідович,ХМЕЛЬНИЦЬКА,ХМЕЛЬНИЦЬКИЙ,"Україна, 29015, Хмельницька обл., місто Хмельницький, ВУЛИЦЯ ЗАРІЧАНСЬКА, будинок 11-Е",49.434774,26.990004


In [169]:
# These areas we will rename to match ukraine.json
df_vaccination_location['registration_area'].unique() 

array(['ЛЬВІВСЬКА', 'ПОЛТАВСЬКА', 'ХМЕЛЬНИЦЬКА', 'СУМСЬКА',
       'МИКОЛАЇВСЬКА', 'ХАРКІВСЬКА', 'ІВАНО-ФРАНКІВСЬКА', 'ЛУГАНСЬКА',
       'ЖИТОМИРСЬКА', 'ВІННИЦЬКА', 'ОДЕСЬКА', 'ЗАПОРІЗЬКА',
       'ДНІПРОПЕТРОВСЬКА', 'ЧЕРНІВЕЦЬКА', 'ЗАКАРПАТСЬКА',
       'КІРОВОГРАДСЬКА', 'ЧЕРНІГІВСЬКА', 'М.КИЇВ', 'КИЇВСЬКА',
       'ЧЕРКАСЬКА', 'ВОЛИНСЬКА', 'РІВНЕНСЬКА', 'ХЕРСОНСЬКА',
       'ТЕРНОПІЛЬСЬКА', 'ДОНЕЦЬКА'], dtype=object)

In [170]:
# Should rename df_vaccination_location['registration_area'] to merge with ukraine.json
geo['NAME_1'].unique()

array(['Cherkasy', 'Chernihiv', 'Chernivtsi', 'Crimea', "Dnipropetrovs'k",
       "Donets'k", "Ivano-Frankivs'k", 'Kharkiv', 'Kherson',
       "Khmel'nyts'kyy", 'Kyiv', 'Kyiv City', 'Kirovohrad', "L'viv",
       "Luhans'k", 'Mykolayiv', 'Odesa', 'Poltava', 'Rivne',
       "Sevastopol'", 'Sumy', "Ternopil'", 'Transcarpathia', 'Vinnytsya',
       'Volyn', 'Zaporizhzhya', 'Zhytomyr'], dtype=object)

In [171]:
df_vaccination_location['care_type'].unique()

array(['Первинна', 'Спеціалізована'], dtype=object)

In [172]:
# We not gonna use this column
df_vaccination_location['property_type'].unique()

array(['Комунальна', 'Приватна (без ФОП)', 'ФОП', 'Державна'],
      dtype=object)

## 2.3 Dataset Info (immunization_vaccine_codes.csv)

#### We discovered that this dataset is not useful and we would not use it later.

In [103]:
df_vaccine_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   vaccine_code  8 non-null      object
 1   vaccine_name  8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [104]:
df_vaccine_info.head()

,vaccine_code,vaccine_name
0,SarsCov2_Pr,Вакцина від коронавірусної хвороби на платформі білкової субодиниці
1,SarsCov2_DNA,Вакцина від коронавірусної хвороби на платформі ДНК
2,SarsCov2_RNA,Вакцина від коронавірусної хвороби на платформі РНК
3,SarsCov2_RVv,Вакцина від коронавірусної хвороби на платформі вірусного вектора зі здатністю до реплікаці
4,SarsCov2_mRNA,Вакцина від коронавірусної хвороби на платформі мРНК


## 3.1 Dataset Cleaning (immunization_covid19_2021_16.11.2021.csv)

In [119]:
df_1 = df_vaccination_entries.copy(deep=True)

In [120]:
# Drop entries with not proper information
df_1.drop(df_1.loc[df_1['status']=='Запис внесено помилково'].index, inplace=True)

In [121]:
# Drop entries with not proper information
df_1.drop(df_1.loc[df_1['patient_age_group']=='Уточнюється'].index, inplace=True)

In [122]:
# Drop entries with not proper information
df_1.drop(df_1.loc[df_1['patient_gender']=='Уточнюється'].index, inplace=True)

In [123]:
# Drop entries with not proper information
df_1.drop(df_1.loc[df_1['not_given']==True].index, inplace=True)

In [124]:
# Drop entries with no information
df_1 = df_1[df_1['dose_sequence'].notna()]

In [125]:
# Drop entries with no information
df_1 = df_1[df_1['series'].notna()]

In [126]:
# Drop entries with no information
df_1 = df_1[df_1['series_doses'].notna()]

In [127]:
# Drop irrelevant columns
df_1.drop('temp_immunization_id', axis=1, inplace=True)
df_1.drop('division_id', axis=1, inplace=True)
df_1.drop('status', axis=1, inplace=True)
df_1.drop('vaccine_code', axis=1, inplace=True)
df_1.drop('immunization_date', axis=1, inplace=True)
df_1.drop('manufacturer', axis=1, inplace=True)
df_1.drop('lot_number', axis=1, inplace=True)
df_1.drop('expiration_date', axis=1, inplace=True)
df_1.drop('dose_quantity_unit', axis=1, inplace=True)
df_1.drop('dose_quantity_value', axis=1, inplace=True)
df_1.drop('vaccination_protocols', axis=1, inplace=True)
df_1.drop('inserted_at', axis=1, inplace=True)
df_1.drop('updated_at', axis=1, inplace=True)
df_1.drop('not_given', axis=1, inplace=True)

In [128]:
def uniform_ages(name):
    name = name.replace("0-4 р.", "0-4")
    name = name.replace("05-10 р.", "5-10")
    name = name.replace("11-15 р.", "11-15")
    name = name.replace("16-20 р.", "16-20")
    name = name.replace("21-25 р.", "21-25")
    name = name.replace("26-30 р.", "26-30")
    name = name.replace("31-35 р.", "31-35")
    name = name.replace("36-40 р.", "36-40")
    name = name.replace("41-45 р.", "41-45")
    name = name.replace("46-50 р.", "46-50")
    name = name.replace("51-55 р.", "51-55")
    name = name.replace("56-60 р.", "56-60")
    name = name.replace("61-65 р.", "61-65")
    name = name.replace("66-70 р.", "66-70")
    name = name.replace("71-75 р.", "71-75")
    name = name.replace("76-80 р.", "76-80")
    name = name.replace("81-85 р.", "81-85")
    name = name.replace("86-00 р.", "86-90")
    name = name.replace("91-95 р.", "91-95")
    name = name.replace("96+ р.", "96+")
    return name

df_1['patient_age_group'] = df_1['patient_age_group'].apply(uniform_ages)

In [129]:
matches_first_dose = ["ПЕР", "пер", "Пер", "1-", "1 етап", "1 й", '1 доза', '1  доза', ' 1', '1етап', ' 1',
                     '1із2', '1 ДОЗА', '1_етап', '1TNFG', "1'", '1й', '1 ЕТАП', 'І-Ї', '1 вакц', '1 із2', 
                     '1`', '1  ДОЗА', 'І дози', '1 із 2', '1тап', '1епізод', '1 вакцинація', 'І-ї', '1  етап',
                     '`1', '1 -й', 'прершої дози', 'І етап', '1 ет', 'введено1дозу', '1 ', 'пеоший', '1доза']
matches_second_dose = ["Дру", "II етап", "2-й етап", "ДРУ", "2 вак", '2 етап', 'IIетап', 'другий ', '2 дози',
                      '2-ої дози', 'другої дози', 'другий', '2 доза', 'друга', '2-етап', 'другої  дози', 'друга доза',
                      'повторний', 'ревакцинація', 'вторинний', '2етап', "2доза" ,'2-й', '2-ї дози', 'ІІ', '2-доза',
                      ]
matches_third_dose = ['третій', '3 етап', 'ТРЕТІЙ', 'ІІІ', '3-й етап', '3']
matches_fourth_dose = ['4 етап', 'четвертий']
matches_fifth_dose = ["5-й етап вакцинації", "п'ятий", 'пятий', '5 етап', 'V', '5 ЕТАП', "П'ЯТИЙ", '5етап',
                     '5-й етап', 'п’ятий', "П'ятий"]


def uniform_series(series):
    if series == "1" or series == "2":
        return series
    if any(x in series for x in matches_first_dose):
        return "1"
    elif any(x in series for x in matches_second_dose):
        return "2"
    elif any(x in series for x in matches_third_dose):
        return "3"
    elif any(x in series for x in matches_fourth_dose):
        return "4"
    elif any(x in series for x in matches_fifth_dose):
        return "5"
    else:
        return series

df_1['series'] = df_1['series'].apply(uniform_series)

In [130]:
available_1 = ['1', '2', '3', '4', '5']
df_1 = df_1[df_1['series'].isin(available_1)]

In [131]:
df_1['series'].value_counts()

1    7271818
5    5848732
2    5013109
3     872216
4     554639
Name: series, dtype: int64

In [132]:
available_2 = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
df_1 = df_1[df_1['series_doses'].isin(available_2)]

In [133]:
df_1 = df_1[df_1['dose_sequence'].isin(available_2)]

In [137]:
def uniform_gender(gender):
    if gender == "Чоловіча":
        return "Male"
    else:
        return "Female"

df_1['patient_gender'] = df_1['patient_gender'].apply(uniform_gender)

In [138]:
df_1.isna().sum()

legal_entity_id      0
patient_age_group    0
patient_gender       0
dose_sequence        0
series               0
series_doses         0
dtype: int64

In [139]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19527207 entries, 1 to 20878296
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   legal_entity_id    object
 1   patient_age_group  object
 2   patient_gender     object
 3   dose_sequence      object
 4   series             object
 5   series_doses       object
dtypes: object(6)
memory usage: 1.0+ GB


In [140]:
df_1.head(3)

,legal_entity_id,patient_age_group,patient_gender,dose_sequence,series,series_doses
1,254ff4d8-5e2f-4db6-9cae-38edabe94729,46-50,Female,2,1,1
5,254ff4d8-5e2f-4db6-9cae-38edabe94729,26-30,Female,2,1,1
6,254ff4d8-5e2f-4db6-9cae-38edabe94729,36-40,Male,2,1,1


## 3.2 Dataset Cleaning (immunization_legal_entities_info.csv)

In [173]:
df_2 = df_vaccination_location.copy(deep=True)

In [174]:
# Drop irrelevant columns
df_2.drop(columns=['legal_entity_name', 'legal_entity_edrpou', 'legal_entity_email', 
                                      'legal_entity_phone', 'legal_entity_owner_name', 'registration_settlement',
                                      'registration_addresses', 'lat', 'lng', 'property_type'], inplace=True)

In [175]:
def uniform_names(name):
    name = name.replace("ЛЬВІВСЬКА", "L'viv")
    name = name.replace("ПОЛТАВСЬКА", "Poltava")
    name = name.replace("ХМЕЛЬНИЦЬКА", "Khmel'nyts'kyy")
    name = name.replace("СУМСЬКА", "Sumy")
    name = name.replace("МИКОЛАЇВСЬКА", "Mykolayiv")
    name = name.replace("ХАРКІВСЬКА", "Kharkiv")
    name = name.replace("ІВАНО-ФРАНКІВСЬКА", "Ivano-Frankivs'k")
    name = name.replace("ЛУГАНСЬКА", "Luhans'k")
    name = name.replace("ЖИТОМИРСЬКА", "Zhytomyr")
    name = name.replace("ВІННИЦЬКА", "Vinnytsya")
    name = name.replace("ОДЕСЬКА", "Odesa")
    name = name.replace("ЗАПОРІЗЬКА", "Zaporizhzhya")
    name = name.replace("ДНІПРОПЕТРОВСЬКА", "Dnipropetrovs'k")
    name = name.replace("ЧЕРНІВЕЦЬКА", "Chernivtsi")
    name = name.replace("ЧЕРНІГІВСЬКА", "Chernihiv")
    name = name.replace("ЧЕРКАСЬКА", "Cherkasy")
    name = name.replace("КІРОВОГРАДСЬКА", "Kirovohrad")
    name = name.replace("КИЇВСЬКА", "Kyiv")
    name = name.replace("М.КИЇВ", "Kyiv City")
    name = name.replace("ТЕРНОПІЛЬСЬКА", "Ternopil'")
    name = name.replace("ВОЛИНСЬКА", "Volyn")
    name = name.replace("РІВНЕНСЬКА", "Rivne")
    name = name.replace("ХЕРСОНСЬКА", "Kherson")
    name = name.replace("ДОНЕЦЬКА", "Donets'k")
    name = name.replace("ЗАКАРПАТСЬКА", "Transcarpathia")
    return name

df_2['registration_area'] = df_2['registration_area'].apply(uniform_names)

In [177]:
df_2.head(3)

,legal_entity_id,care_type,registration_area
0,ad229a7d-9bfe-4cf9-9196-4c7fc91f8c5e,Первинна,L'viv
1,ed60aaef-83c5-4a5f-988c-e906822d3731,Первинна,Poltava
2,2e756a68-36ff-4469-95db-0d367b1fbbdb,Первинна,Khmel'nyts'kyy


## 4. Datasets Merging

In [184]:
df_merged = pd.merge(df_1, df_2, left_on="legal_entity_id", right_on="legal_entity_id", how="left")

In [185]:
# Drop irrelevant column
df_merged.drop('legal_entity_id', axis=1, inplace=True)

In [186]:
df_merged = df_merged.rename(columns = {'patient_age_group': 'Age_group'}, inplace = False)
df_merged = df_merged.rename(columns = {'patient_gender': 'Gender'}, inplace = False)
df_merged = df_merged.rename(columns = {'dose_sequence': 'Vaccination_sequence_number'}, inplace = False)
df_merged = df_merged.rename(columns = {'series': 'Vaccination_phase'}, inplace = False)
df_merged = df_merged.rename(columns = {'series_doses': 'Vaccination_needed_doses'}, inplace = False)
df_merged = df_merged.rename(columns = {'care_type': 'Medical_care'}, inplace = False)
df_merged = df_merged.rename(columns = {'registration_area': 'Region'}, inplace = False)

In [187]:
def uniform_hospital(hospital):
    if hospital == "Первинна":
        return "First aid"
    else:
        return "Specialized aid"

df_merged['Medical_care'] = df_merged['Medical_care'].apply(uniform_hospital)

In [188]:
df_merged.head()

,Age_group,Gender,Vaccination_sequence_number,Vaccination_phase,Vaccination_needed_doses,Medical_care,Region
0,46-50,Female,2,1,1,First aid,Chernihiv
1,26-30,Female,2,1,1,First aid,Chernihiv
2,36-40,Male,2,1,1,First aid,Chernihiv
3,46-50,Male,2,1,1,First aid,Chernihiv
4,41-45,Male,1,1,2,First aid,Zhytomyr


In [189]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19527207 entries, 0 to 19527206
Data columns (total 7 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   Age_group                    object
 1   Gender                       object
 2   Vaccination_sequence_number  object
 3   Vaccination_phase            object
 4   Vaccination_needed_doses     object
 5   Medical_care                 object
 6   Region                       object
dtypes: object(7)
memory usage: 1.2+ GB


In [190]:
df_merged.isna().sum()

Age_group                      0
Gender                         0
Vaccination_sequence_number    0
Vaccination_phase              0
Vaccination_needed_doses       0
Medical_care                   0
Region                         0
dtype: int64

## 5. Saving Cleaned Dataset

In [193]:
# Run to save cleaned data to .csv file
df_merged.to_csv('./data/cleaned_covid_data.csv', index=False)